# Project Title
### Data Engineering Capstone Project
## Sebastian Baeza October - 2019
#### Project Summary

The data sources will be aggregated with Spark SQL (an instance is available by project template), and pandas will be used to analyze the data.

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd
import os
import glob
from us_state_abbrev import state_udf, abbrev_state, abbrev_state_udf,city_code_udf,city_codes
from immigration_codes import country_udf
from pyspark.sql import SparkSession, SQLContext, GroupedData
from pyspark.sql.functions import *

In [2]:
#Build spark session
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()

In [3]:
#Build SQL context object
sqlContext = SQLContext(spark)

### Step 1: Scope the Project and Gather Data

#### Scope 
This project will get data from all available sources in the project template and create dimension and fact tables to show some insights of immigration in the USA.

Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 




#### Loading data in dataframes

In [8]:
df_demog = pd.read_csv("us-cities-demographics.csv", sep=";", header=0)

df_airport = pd.read_csv("airport-codes_csv.csv", sep=",", header=0)

df_temp = pd.read_csv("GlobalLandTemperaturesByState.csv", sep=",", header=0)

df_immi = pd.read_sas("../../data/18-83510-I94-Data-2016/i94_dec16_sub.sas7bdat")

#### Loading data in Spark

In [5]:
# Read in the data here
spark_demographics=spark.read.format("csv").option("header", "true").option("delimiter", ";").load("us-cities-demographics.csv")

spark_airport=spark.read.format("csv").option("header", "true").load("airport-codes_csv.csv")

spark_temperature=spark.read.format("csv").option("header", "true").load("GlobalLandTemperaturesByState.csv")

spark_sas_data=spark.read.format('com.github.saurfang.sas.spark').load("../../data/18-83510-I94-Data-2016/i94_dec16_sub.sas7bdat")

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

First, let's take a view about data sources

#### U.S cities demographics

In [6]:
df_demog.head()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


In [7]:
df_demog.describe()

,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,Count
count,2891.000000,2.888000e+03,2.888000e+03,2.891000e+03,2878.000000,2.878000e+03,2875.000000,2.891000e+03
mean,35.494881,9.732843e+04,1.017696e+05,1.989668e+05,9367.832523,4.065360e+04,2.742543,4.896377e+04
std,4.401617,2.162999e+05,2.315646e+05,4.475559e+05,13211.219924,1.557491e+05,0.433291,1.443856e+05
min,22.900000,2.928100e+04,2.734800e+04,6.321500e+04,416.000000,8.610000e+02,2.000000,9.800000e+01
25%,32.800000,3.928900e+04,4.122700e+04,8.042900e+04,3739.000000,9.224000e+03,2.430000,3.435000e+03
50%,35.300000,5.234100e+04,5.380900e+04,1.067820e+05,5397.000000,1.882200e+04,2.650000,1.378000e+04
75%,38.000000,8.664175e+04,8.960400e+04,1.752320e+05,9368.000000,3.397175e+04,2.950000,5.444700e+04
max,70.500000,4.081698e+06,4.468707e+06,8.550405e+06,156961.000000,3.212500e+06,4.980000,3.835726e+06


#### comments about demographic data


#### Airport codes

In [16]:
df_airport.head()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"


In [17]:
df_airport.describe()

,elevation_ft
count,48069.000000
mean,1240.789677
std,1602.363459
min,-1266.000000
25%,205.000000
50%,718.000000
75%,1497.000000
max,22000.000000


comments about airplane data

In [9]:
df_immi.head()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,46.0,2016.0,12.0,129.0,129.0,b'HOU',20789.0,1.0,b'TX',20802.0,...,NaN,b'M',1970.0,b'05262018',b'M',NaN,b'RS',9.755414e+10,b'7715',b'E2'
1,56.0,2016.0,12.0,245.0,245.0,b'NEW',20789.0,1.0,b'OH',20835.0,...,NaN,b'M',1988.0,b'D/S',b'F',NaN,b'CA',9.062372e+10,b'819',b'F1'
2,67.0,2016.0,12.0,512.0,512.0,b'PEV',20789.0,2.0,b'MD',20794.0,...,NaN,b'M',1968.0,b'06012017',b'M',b'5920',NaN,8.010503e+10,NaN,b'B2'
3,68.0,2016.0,12.0,512.0,512.0,b'PEV',20789.0,2.0,b'FL',20792.0,...,NaN,b'M',1970.0,b'06012017',b'F',b'5920',NaN,8.010511e+10,NaN,b'B2'
4,69.0,2016.0,12.0,512.0,512.0,b'PEV',20789.0,2.0,b'HI',20792.0,...,NaN,b'M',1968.0,b'06012017',b'M',b'5920',NaN,8.010511e+10,NaN,b'B2'


In [10]:
df_immi.describe()

,cicid,i94yr,i94mon,i94cit,i94res,arrdate,i94mode,depdate,i94bir,i94visa,count,biryear,admnum
count,3.432990e+06,3432990.0,3432990.0,3.432021e+06,3.432990e+06,3.432990e+06,3.432990e+06,3.218202e+06,3.432297e+06,3.432990e+06,3432990.0,3.432297e+06,3.432990e+06
mean,3.661998e+06,2016.0,12.0,3.459631e+02,3.410846e+02,2.080512e+04,1.065441e+00,2.082203e+04,3.781565e+01,1.949797e+00,1.0,1.978184e+03,2.191541e+10
std,2.064380e+06,0.0,0.0,2.188854e+02,2.166048e+02,8.643925e+00,4.140615e-01,3.735567e+01,1.869041e+01,3.046412e-01,0.0,1.869041e+01,1.338326e+10
min,6.000000e+00,2016.0,12.0,0.000000e+00,1.010000e+02,2.078900e+04,1.000000e+00,-1.197200e+04,-1.000000e+00,1.000000e+00,1.0,1.907000e+03,0.000000e+00
25%,1.873762e+06,2016.0,12.0,1.350000e+02,1.350000e+02,2.079800e+04,1.000000e+00,2.080700e+04,2.400000e+01,2.000000e+00,1.0,1.965000e+03,1.665601e+10
50%,3.745504e+06,2016.0,12.0,2.510000e+02,2.450000e+02,2.080600e+04,1.000000e+00,2.081900e+04,3.700000e+01,2.000000e+00,1.0,1.979000e+03,1.803344e+10
75%,5.414167e+06,2016.0,12.0,5.820000e+02,5.820000e+02,2.081200e+04,1.000000e+00,2.082600e+04,5.100000e+01,2.000000e+00,1.0,1.992000e+03,1.943026e+10
max,7.318723e+06,2016.0,12.0,9.990000e+02,7.490000e+02,2.081900e+04,9.000000e+00,2.098400e+04,1.090000e+02,3.000000e+00,1.0,2.017000e+03,9.999752e+10


comments about immi

#### US Temperature

#### Cleaning Steps
Document steps necessary to clean the data

In [7]:
# Performing cleaning tasks here





Immigration Data by State with Origin

In [9]:
# 1.- Remove nulls values
# 2.- Convert i94res codes to country of origin
# 3.- Filter out NULLS
# 4.- Run country_udf function to show state names
# country_udf, abbrev_state_udf and city_code_udf were created with data from i94 SAS labels Descriptions file.
i94_sas_data=spark_sas_data.filter(spark_sas_data.i94addr.isNotNull())\
.filter(spark_sas_data.i94res.isNotNull())\
.filter(col("i94addr").isin(list(abbrev_state.keys())))\
.filter(col("i94port").isin(list(city_codes.keys())))\
.withColumn("origin_country",country_udf(spark_sas_data["i94res"]))\
.withColumn("dest_state_name",abbrev_state_udf(spark_sas_data["i94addr"]))\
.withColumn("i94yr",col("i94yr").cast("integer"))\
.withColumn("i94mon",col("i94mon").cast("integer"))\
.withColumn("city_port_name",city_code_udf(spark_sas_data["i94port"]))

new_i94_sas_data=i94_sas_data.select("cicid",col("i94yr").alias("year"),col("i94mon").alias("month"),\
                             "origin_country","i94port","city_port_name",col("i94addr").alias("state_code"),"dest_state_name")

In [10]:
new_i94_sas_data.show(5)

+-----+----+-----+--------------+-------+------------------+----------+---------------+
|cicid|year|month|origin_country|i94port|    city_port_name|state_code|dest_state_name|
+-----+----+-----+--------------+-------+------------------+----------+---------------+
| 46.0|2016|   12|         SPAIN|    HOU|HOUSTON           |        TX|          Texas|
| 56.0|2016|   12|    CHINA, PRC|    NEW|NEWARK/TETERBORO  |        OH|           Ohio|
| 67.0|2016|   12|       BAHAMAS|    PEV|PORT EVERGLADES   |        MD|       Maryland|
| 68.0|2016|   12|       BAHAMAS|    PEV|PORT EVERGLADES   |        FL|        Florida|
| 69.0|2016|   12|       BAHAMAS|    PEV|PORT EVERGLADES   |        HI|         Hawaii|
+-----+----+-----+--------------+-------+------------------+----------+---------------+
only showing top 5 rows



In [11]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()
df_spark =spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')


In [12]:
#write to parquet
df_spark.write.parquet("sas_data")
df_spark=spark.read.parquet("sas_data")

AnalysisException: 'path file:/home/workspace/sas_data already exists.;'

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.